# Forma Fitness Chatbot - Kaggle Training

**FREE GPU Training on Kaggle**

1. Upload your `fitness-sharegpt.json` to Kaggle as a dataset
2. Enable GPU: Settings -> Accelerator -> GPU T4 x2
3. Run all cells

In [ ]:
# Install Unsloth
!pip install unsloth
!pip install --no-deps trl peft accelerate bitsandbytes

In [ ]:
import torch
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
from unsloth import FastLanguageModel
from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments
import json

# Load model with 4-bit quantization
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen2.5-3B-Instruct-bnb-4bit",
    max_seq_length=2048,
    load_in_4bit=True,
)

# Add LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    use_gradient_checkpointing="unsloth",
)

In [ ]:
# Load your dataset (update path if needed)
# If uploaded as Kaggle dataset: /kaggle/input/your-dataset-name/fitness-sharegpt.json
DATA_PATH = "/kaggle/input/fitness-dataset/fitness-sharegpt.json"

with open(DATA_PATH, "r") as f:
    raw_data = json.load(f)

SYSTEM = "You are Forma AI, a knowledgeable fitness assistant."

def format_conv(ex):
    text = f"<|im_start|>system\n{SYSTEM}<|im_end|>\n"
    for m in ex["conversations"]:
        role = "user" if m["from"] == "human" else "assistant"
        text += f"<|im_start|>{role}\n{m['value']}<|im_end|>\n"
    return {"text": text}

dataset = Dataset.from_list(raw_data).map(format_conv)
print(f"Training samples: {len(dataset)}")

In [ ]:
# Train
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        optim="adamw_8bit",
        output_dir="outputs",
    ),
)

trainer.train()

In [ ]:
# Save as GGUF for llama.cpp
model.save_pretrained_gguf("forma-fitness", tokenizer, quantization_method="q4_k_m")
print("Model saved! Download the .gguf file from the output.")

In [ ]:
# Test it
FastLanguageModel.for_inference(model)

messages = [{"role": "user", "content": "How do I build muscle?"}]
inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
outputs = model.generate(inputs, max_new_tokens=200, temperature=0.7)
print(tokenizer.decode(outputs[0]))